In [2]:
# Modelo de regresión para dependencia espacial tipo SAR

import os
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import box
from rasterstats import zonal_stats
from libpysal.weights import Queen
from spreg import ML_Lag

ruta_geoformas = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\SHP\Inventario_geoformas_karsticas_Dunita.shp"
ruta_contorno  = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\SHP\Contorno_Dunita.shp"
ruta_dem       = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\Raster\dem_clip.tif"
ruta_slope     = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\Raster\pendiente.tif"
ruta_twi       = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\Raster\TWI.tif"
ruta_cover     = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\SHP\Cobertura_final.shp"
ruta_drain     = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\SHP\Drenajes_clip_POT_final.shp"
ruta_fault     = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\SHP\Fallas_lineam.shp"

gdf_points  = gpd.read_file(ruta_geoformas)
gdf_contour = gpd.read_file(ruta_contorno)

# Generar cuadrícula de 100x100 metros
cell_size = 100
minx, miny, maxx, maxy = gdf_contour.total_bounds
xs = np.arange(minx, maxx, cell_size)
ys = np.arange(miny, maxy, cell_size)
polygons = [box(x, y, x+cell_size, y+cell_size) for x in xs for y in ys]
grid = gpd.GeoDataFrame({'geometry': polygons}, crs=gdf_contour.crs)
grid = gpd.clip(grid, gdf_contour)

# Conteo de puntos (variable dependiente)
join = gpd.sjoin(grid, gdf_points, how='left', predicate='intersects')
counts = join.groupby(join.index).size()
grid['count'] = counts.reindex(grid.index).fillna(0).astype(int)

# Extraer estadísticas zonales de los rasters
grid['dem_mean']   = [s['mean'] if s and s['mean'] is not None else np.nan for s in zonal_stats(grid, ruta_dem,   stats=['mean'], nodata=-9999)]
grid['slope_mean'] = [s['mean'] if s and s['mean'] is not None else np.nan for s in zonal_stats(grid, ruta_slope, stats=['mean'], nodata=-9999)]
grid['twi_mean']   = [s['mean'] if s and s['mean'] is not None else np.nan for s in zonal_stats(grid, ruta_twi,   stats=['mean'], nodata=-9999)]

# Cobertura y calcular distancias
cov_gdf  = gpd.read_file(ruta_cover)[['d_N3_COBER','geometry']].to_crs(grid.crs)
centroids= grid.copy()
centroids['geometry'] = centroids.centroid
cov_join = gpd.sjoin(centroids, cov_gdf, how='left', predicate='within')
grid['cover'] = cov_join['d_N3_COBER'].fillna('None')

drains = gpd.read_file(ruta_drain).to_crs(grid.crs)
faults = gpd.read_file(ruta_fault).to_crs(grid.crs)

union_drains = drains.geometry.union_all()
union_faults = faults.geometry.union_all()

grid['dist_drain'] = centroids.geometry.distance(union_drains)
grid['dist_fault'] = centroids.geometry.distance(union_faults)

# Limpiar datos: eliminar filas con valores nulos
grid_clean = grid.dropna()


# Selección de variables y preparación de matrices para el modelo SAR 

# Variables continuas 
vars_continuas = ['dem_mean', 'slope_mean', 'twi_mean', 'dist_drain', 'dist_fault']

# Variables categóricas significativas
vars_categoricas_sig = [
    'Bosque fragmentado',
    'Mosaico de cultivos y espacios naturales'
]

# Crear variables dummy solo para las coberturas seleccionadas
df_sar = grid_clean.copy()
df_dummies = pd.get_dummies(df_sar['cover'], prefix='cov')

# Nos aseguramos de que los nombres de las columnas existan antes de usarlos
dummies_a_usar = [f"cov_{cat}" for cat in vars_categoricas_sig if f"cov_{cat}" in df_dummies.columns]

# Unir las dummies seleccionadas al GeoDataFrame
df_sar = df_sar.join(df_dummies[dummies_a_usar])

# Lista final de variables explicativas (X)
X_vars = vars_continuas + dummies_a_usar

# Preparar matrices para spreg
y = df_sar['count'].values.reshape(-1, 1)
X = df_sar[X_vars].astype(float).values

# Ajuste del Modelo 

# Matriz de pesos espaciales (contigüidad tipo Queen)
w = Queen.from_dataframe(df_sar, use_index=False)
w.transform = 'r' # Estandarizar por filas

# Ajustar el modelo SAR
model_sar = ML_Lag(
    y, X, w=w,
    name_y='count',
    name_x=X_vars,
    name_w='Queen'
)

# ───────────────────────────────────────────────────────────────────────────────
# --- 6. Imprimir resumen del modelo ---
print(model_sar.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :       Queen
Dependent Variable  :       count                Number of Observations:        2426
Mean dependent var  :      1.4596                Number of Variables   :           9
S.D. dependent var  :      1.7774                Degrees of Freedom    :        2417
Pseudo R-squared    :      0.2938
Spatial Pseudo R-squared:  0.1093
Log likelihood      :  -4496.4363
Sigma-square ML     :      2.2615                Akaike info criterion :    9010.873
S.E of regression   :      1.5038                Schwarz criterion     :    9063.019

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
---------------------------------------------------------------